# Trends and Analysis of eCommerce Data

Francesca Barrios
frba7936@colorado.edu

Laura Brown
labr1398@colorado.edu

Seth Ely  
seel6470@colorado.edu

Darby Hansen  
daha2377@colorado.edu 

## Data Cleaning

In [7]:
# load datasets and merge as single pandas dataframe



In [8]:
# parse date and time from event_time feature



In [9]:
# fill in missing values in category code



In [10]:
# remove duplicate entries



### Which brands, items, or item categories exhibit the highest volume of sales during the specified period? 

In [11]:
# filter by purchased


# group by brand


# calculate sum of price


# group by item


# calculate sum of price


# group by category


# calculate sum of price


# vizualization (table, graph) for brand/item/category



### Are there discernible trends in sales performance across different brands and categories? 

In [12]:
# filter by purchased


# filter by top brands and/or categories


# graph brands/categories over time
# (day/week/month whichever works best considering the large amount of entries)

### Among the features of brands, items, or item categories, which ones demonstrate the highest purchases per view ratio? 

In [13]:
# groupby brand


# Calculate number purchased / number of entries


# groupby item


# Calculate number purchased / number of entries


# groupby category


# Calculate number purchased / number of entries

### What factors contribute to the rate of purchases per view of these brands, items, or item categories? 

In [14]:
# Interpret

### Can brand type lead to customer retention (customer’s purchase frequency after initial purchase)? 

In [15]:
# groupby user_id


# calculate percentage of purchases that are one brand over total purchases

### Can seasonal patterns be identified within the 7-month window of the dataset? 

In [16]:
# filter by purchased


# graph brands/categories over time
# (day/week/month whichever works best considering the large amount of entries)

### How do seasonal variations impact the sales and purchasing behaviors of consumers across individual brands and categories? 

### Can we find any trending patterns within our item sets outside of seasonal trend behavior – which can be used to predict popularity purchase windows? 

## Evaluation

### Accuracy

In [17]:
# Predictive model with target class event_type (purchased = 1, view/remove from cart = 0)

In [18]:
# Cluster algorithm for-you recommendations

### Relevance

_discuss findings in terms of real-word needs_

### Interpretability

_discuss visualizations and how understandable the findings are to the general public_